In [5]:
from Functions import *

In [6]:
# get the data
filname = 'fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [7]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

7


In [8]:
N, D = X.shape
X = X.reshape(N, 48, 48, 1)
cascade = load_cascade_classifier_xml()
print(X.shape)

(35887, 48, 48, 1)


In [9]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [10]:
path_model='model_filter.h5' # save model at this location after each epoch
model=my_model() # create the model
model.load_weights(path_model)
# fit the model
# h=model.fit(x=X_train,     
#             y=y_train, 
#             batch_size=64, 
#             epochs=20, 
#             verbose=1, 
#             validation_data=(X_test,y_test),
#             shuffle=True,
#             callbacks=[
#                 ModelCheckpoint(filepath=path_model),
#             ]
#             )

In [11]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

[0 1 2 3 4 5 6]


In [12]:
y_pred=model.predict(X_test)
y_test.shape


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2020-10-29 11:02:03,053 WARNING From /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training_v1.py:2070: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


(3589, 7)

In [13]:
y_train.shape

(32298, 7)

In [14]:

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")
sfps = 0
  
fps = 0
while True:
    ret, frame = cap.read()
    baseframe = frame
    humans = e.inference(baseframe,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=resize_out_ratio)

    #logger.debug('postprocess+')
    image = TfPoseEstimator.draw_humans(baseframe, humans, imgcopy=False)
    coords,faces,faceframe = find_faces_in_img(frame,cascade)
    if len(faces) < 1:
        fps=time()
        FPS=1/(fps-sfps)
        sfps = fps 

        FPS = 'FPS:'+str(round(FPS,ndigits=3))
        maketextaboveface(baseframe,FPS,(10,70))
        cv2.imshow('feed', baseframe)
        
        c = cv2.waitKey(1)
        if c == 27:
            break

        continue
    for i in range(0,len(faces)):
        face = faces[i]
        frame = prep_image(face)

        prediction = predict_emotion(frame,model)

        text = 'Prediction = '+str(prediction)
        maketextaboveface(baseframe,text,(coords[i][1],coords[i][0]))
    fps=time()
    FPS=1/(fps-sfps)
    sfps = fps
    FPS = 'FPS:'+str(round(FPS,ndigits=3))+' & We Found '+str(len(faces))+ ' Face(s)!'
    maketextaboveface(baseframe,FPS,(10,70))
    cv2.imshow('feed', baseframe)
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import sys
import time
import logging
import numpy as np
import cv2

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

logger = logging.getLogger('TfPoseEstimator-WebCam')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")
camera = 0
resize = '432x368'     # resize images before they are processed
resize_out_ratio = 4.0 # resize heatmaps before they are post-processed
model = 'mobilenet_thin'
show_process = False
tensorrt = False       # for tensorrt process
# logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
w, h = model_wh(resize)
if w > 0 and h > 0:
    e = TfPoseEstimator(get_graph_path(model), target_size=(w, h), trt_bool=False)
else:
    e = TfPoseEstimator(get_graph_path(model), target_size=(432, 368), trt_bool=False)
humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=resize_out_ratio)

    #logger.debug('postprocess+')
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)


IndentationError: unexpected indent (<ipython-input-4-cd93a8853384>, line 38)

In [ ]:
# 